# Project - Cars classification based on Type using Logistic Regression

# Read Data

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import pandas as pd
A = pd.read_csv("C:/Users/ABC/Downloads/D0wnloded/Cars93.csv")

In [5]:
A.columns=['id', 'Manufacturer', 'Model', 'Type', 'MinPrice', 'Price',
       'MaxPrice', 'MPGcity', 'MPGhighway', 'AirBags', 'DriveTrain',
       'Cylinders', 'EngineSize', 'Horsepower', 'RPM', 'Revpermile',
       'Mantransavail', 'Fueltankcapacity', 'Passengers', 'Length',
       'Wheelbase', 'Width', 'Turncircle', 'Rearseatroom', 'Luggageroom',
       'Weight', 'Origin', 'Make']

# Preview

In [8]:
A.head()

,id,Manufacturer,Model,Type,MinPrice,Price,MaxPrice,MPGcity,MPGhighway,AirBags,...,Passengers,Length,Wheelbase,Width,Turncircle,Rearseatroom,Luggageroom,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


# Missing Data Treatment

In [9]:
from PM8wd import replacer
replacer(A)

In [11]:
#A.isna().sum()

# Drop unwanted Colummns

In [12]:
A = A.drop(labels=["id","Make","Model"],axis=1)

# Define X and Y

In [13]:
X = A.drop(labels=["Type"],axis=1)
Y = A[["Type"]]

# Feature selection based on EDA

In [14]:
def ANOVA(df,cat,con):
    from pandas import DataFrame
    from statsmodels.api import OLS
    from statsmodels.formula.api import ols
    rel = con + " ~ " + cat
    model = ols(rel,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    Q = DataFrame(anova_results)
    a = Q['PR(>F)'][cat]
    return round(a,3)


In [15]:
imp_cols = []
for i in X.columns:
    if(X[i].dtypes!="object"):
        x = ANOVA(A,"Type",i)
        if(x < 0.05):
            print("Type vs ",i,"-------->",x)
            imp_cols.append(i)
        

Type vs  MinPrice --------> 0.0
Type vs  Price --------> 0.0
Type vs  MaxPrice --------> 0.0
Type vs  MPGcity --------> 0.0
Type vs  MPGhighway --------> 0.0
Type vs  EngineSize --------> 0.0
Type vs  Horsepower --------> 0.0
Type vs  RPM --------> 0.0
Type vs  Revpermile --------> 0.0
Type vs  Fueltankcapacity --------> 0.0
Type vs  Passengers --------> 0.0
Type vs  Length --------> 0.0
Type vs  Wheelbase --------> 0.0
Type vs  Width --------> 0.0
Type vs  Turncircle --------> 0.0
Type vs  Rearseatroom --------> 0.0
Type vs  Luggageroom --------> 0.0
Type vs  Weight --------> 0.0


In [17]:
#imp_cols

In [18]:
from scipy.stats import chi2_contingency
def chisquare(df,cat1,cat2):
    import pandas as pd
    ct = pd.crosstab(df[cat1],df[cat2])
    a,b,c,d = chi2_contingency(ct)
    return b

In [19]:
for i in X.columns:
    if(X[i].dtypes=="object"):
        x = chisquare(A,"Type",i)
        if(x < 0.05):
            print("Type vs ",i,"-->",x)
            imp_cols.append(i)

Type vs  AirBags --> 0.00022621310210534361
Type vs  DriveTrain --> 0.00012480179759447616
Type vs  Cylinders --> 1.674244316924938e-07
Type vs  Mantransavail --> 3.661469229213669e-10
Type vs  Origin --> 0.015110051037674484


# Preprocessing

In [20]:
from PM8wd import preprocessing
Xnew = preprocessing(X[imp_cols])

In [21]:
Xnew

,MinPrice,Price,MaxPrice,MPGcity,MPGhighway,EngineSize,Horsepower,RPM,Revpermile,Fueltankcapacity,...,Cylinders_3,Cylinders_4,Cylinders_5,Cylinders_6,Cylinders_8,Cylinders_rotary,Mantransavail_No,Mantransavail_Yes,Origin_USA,Origin_non-USA
0,-0.485787,-0.375720,-0.282465,0.471312,0.360925,-0.841022,-0.073484,1.717489,1.129530,-1.062184,...,0,1,0,0,0,0,0,1,0,1
1,1.388017,1.497844,1.531409,-0.781032,-0.770514,0.515869,1.078322,0.369586,0.005661,0.409445,...,0,0,0,1,0,0,0,1,0,1
2,1.008658,0.998227,0.948052,-0.423219,-0.581941,0.128186,0.540813,0.369586,-0.105713,0.072197,...,0,0,0,1,0,0,0,1,0,1
3,1.571949,1.893374,2.069191,-0.602126,-0.581941,0.128186,0.540813,0.369586,0.410659,1.359872,...,0,0,0,1,0,0,0,1,0,1
4,0.755752,1.091905,1.303535,-0.065407,0.172352,0.806631,1.231897,0.706562,0.430909,1.359872,...,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,-0.060445,0.019810,0.073018,-0.959938,-1.524806,-0.162577,-0.668585,-1.315292,1.180155,1.359872,...,0,0,1,0,0,0,0,1,0,1
89,0.054512,0.051036,0.045673,-0.244313,0.172352,-0.647181,-0.188665,0.875050,0.714407,0.562740,...,0,1,0,0,0,0,0,1,0,1
90,0.663786,0.394523,0.164167,-0.781032,-0.770514,0.128186,0.655993,0.875050,0.106911,0.562740,...,0,0,0,1,0,0,0,1,0,1
91,0.537333,0.332071,0.145937,-0.244313,-0.204794,-0.356418,-0.572601,0.201098,-0.237337,-0.265051,...,0,1,0,0,0,0,0,1,0,1


# Split the Data in training and testing set

In [22]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,random_state=21,test_size=0.2)

# Create Logestic Regression model

In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(xtrain,ytrain)

pred_ts = model.predict(xtest)
pred_tr = model.predict(xtrain)

from sklearn.metrics import accuracy_score

tr_acc = accuracy_score(ytrain,pred_tr)
ts_acc = accuracy_score(ytest,pred_ts)

In [24]:
tr_acc

1.0

In [25]:
ts_acc

0.9473684210526315

# Confusion_matrix

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest,pred_ts)

array([[1, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 0, 0],
       [0, 0, 4, 0, 0, 0],
       [1, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 2]], dtype=int64)

In [27]:
ytest['pred']=pred_ts

In [28]:
ytest.sort_values(by=["Type","pred"])

,Type,pred
91,Compact,Compact
21,Large,Large
17,Large,Large
19,Large,Large
10,Midsize,Midsize
3,Midsize,Midsize
1,Midsize,Midsize
36,Midsize,Midsize
23,Small,Compact
82,Small,Small
